In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd() , '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from src.modelling.inference import detect_human_poses_in_video

video_path = f"{project_root}/data/raw/germany-vs-spain-cutted.mp4"
output_path = f"{project_root}/data/pose_detected/germany-vs-spain-cutted.mp4"

In [2]:
results = detect_human_poses_in_video(video_path, output_path)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/145) /home/denis/Projects/var-ai/data/raw/germany-vs-spain-cutted.mp4: 384x640 9 persons, 101.3ms
video 1/1 (frame 2/145) /home/denis/Projects/var-ai/data/raw/germany-vs-spain-cutted.mp4: 384x640 8 persons, 70.6ms
video 1/1 (frame 3/145) /home/denis/Projects/var-ai/data/raw/germany-vs-spain-cutted.mp4: 384x640 8 persons, 57.3ms
video 1/1 (frame 4/145) /home/denis/Projects/var-ai/data/raw/germany-vs-spain-cutted.mp4: 384x640 7 persons

In [5]:
import pandas as pd

output_path_csv = f"{project_root}/data/coordinates/"

keypoint_indices = {
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,  # left hand
    'right_wrist': 10  # right hand
}

# Create a list to store all frames' data
all_frames_data = []

# Process each frame's results
for frame_idx, result in enumerate(results):
    if result.keypoints is not None:
        # Get keypoints data
        keypoints = result.keypoints.data
        
        # For each person detected in the frame
        for person_idx, person_kpts in enumerate(keypoints):
            # Extract only the needed keypoints (hands and elbows)
            for part_name, part_idx in keypoint_indices.items():
                if part_idx < len(person_kpts):
                    x, y, conf = person_kpts[part_idx].tolist()
                    
                    # Add data to list
                    all_frames_data.append({
                        'frame': frame_idx,
                        'person_id': person_idx,
                        'body_part': part_name,
                        'x': x,
                        'y': y,
                        'confidence': conf
                    })

# Create DataFrame and save to CSV
if all_frames_data:
    df = pd.DataFrame(all_frames_data)
    
    # Create file name based on input video
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_file = os.path.join(output_path_csv, f"{video_name}_hand_elbow_keypoints.csv")
    
    df.to_csv(csv_file, index=False)
    print(f"Keypoints data saved to {csv_file}")

Keypoints data saved to /home/denis/Projects/var-ai/data/coordinates/germany-vs-spain-cutted_hand_elbow_keypoints.csv
